In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from tensorflow import keras 
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.mobilenet import preprocess_input

import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from tensorflow.keras.applications.mobilenet import MobileNet

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
data_dir = '/home/atik/Documents/Summer_1/data/'

In [16]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(224, 224),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', shuffle=False,
    target_size=(224, 224),
    batch_size=64
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


In [17]:
from collections import Counter
counter = Counter(train_generator.classes)
print(len(counter))
print(train_generator)

256


# Model

In [18]:
model = MobileNet(weights=None, classes=len(counter),classifier_activation="softmax")
model.count_params()

3491264

# Training

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, nesterov=True), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [20]:
model.fit(
    train_generator, 
    steps_per_epoch=400, epochs=30, verbose=1,
    callbacks=[
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, epsilon=0.007),
        EarlyStopping(monitor='val_accuracy', patience=4, min_delta=0.01)
    ],
    validation_data=val_generator, validation_steps=80, workers=4
)

Epoch 1/30


2022-05-24 10:39:14.373039: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 36.75MiB (rounded to 38535168)requested by op SameWorkerRecvDone
Current allocation summary follows.
2022-05-24 10:39:14.373112: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-05-24 10:39:14.373118: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 119, Chunks in use: 118. 29.8KiB allocated for chunks. 29.5KiB in use in bin. 9.3KiB client-requested in use in bin.
2022-05-24 10:39:14.373122: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 49, Chunks in use: 49. 24.8KiB allocated for chunks. 24.8KiB in use in bin. 24.5KiB client-requested in use in bin.
2022-05-24 10:39:14.373125: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 57, Chunks in use: 57. 59.8KiB allocated for chunks. 59.8KiB in use in bin. 57.5K

ResourceExhaustedError:  SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;8cff56f7e5ae41f8;/job:localhost/replica:0/task:0/device:GPU:0;edge_142_IteratorGetNext;0:0
	 [[{{node IteratorGetNext/_2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_9881]

Function call stack:
train_function


# Loss/epoch plots

In [ ]:
plt.plot(model.history.history['loss'], label='train');
plt.plot(model.history.history['val_loss'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('logloss');

In [ ]:
plt.plot(model.history.history['accuracy'], label='train');
plt.plot(model.history.history['val_accuracy'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('accuracy');

In [ ]:
plt.plot(model.history.history['top_k_categorical_accuracy'], label='train');
plt.plot(model.history.history['val_top_k_categorical_accuracy'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('top5_accuracy');

# Results

In [ ]:
model.evaluate_generator(val_generator, 80)